<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: <https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv> 

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [159]:
import pandas as pd
import numpy as np

url = 'https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv'
df = pd.read_csv(url)
print(df.shape)
df.head()

(7043, 21)


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [160]:
def wrangle_data(df):
    """ Wrangle function for cleaning and factorizing dataframe"""

    # Dropping unecessary column
    df.drop(['customerID'], axis=1, inplace=True)

    # Factorizing and renaming columns
    col_list = ['MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 
                'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 
                'Contract', 'PaymentMethod'] 

    for col in col_list:
        df[col] = pd.factorize(df[col])[0] + 1

    df = pd.get_dummies(
        df, prefix=['gender', 'Partner', 'Dependents', 'PhoneService', 'PaperlessBilling', 'Churn'],
        columns=['gender', 'Partner', 'Dependents', 'PhoneService', 'PaperlessBilling', 'Churn'],
        drop_first=True)

    df = df.rename(
        columns={'gender_Male':'gender', 'Partner_Yes':'Partner', 
        'Dependents_Yes':'Dependents', 'PhoneService_Yes':'PhoneService', 
        'PaperlessBilling_Yes':'PaperlessBilling', 'Churn_Yes':'Churn'})

    # Setting 'TotalCharges' to float instead of object
    df['TotalCharges'] = pd.to_numeric(df['TotalCharges'],errors='coerce')

    return df

In [161]:
df = wrangle_data(df)
df.head()

,SeniorCitizen,tenure,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaymentMethod,MonthlyCharges,TotalCharges,gender,Partner,Dependents,PhoneService,PaperlessBilling,Churn
0,0,1,1,1,1,1,1,1,1,1,1,1,29.85,29.85,0,1,0,0,1,0
1,0,34,2,1,2,2,2,1,1,1,2,2,56.95,1889.50,1,0,0,1,0,0
2,0,2,2,1,2,1,1,1,1,1,1,2,53.85,108.15,1,0,0,1,1,1
3,0,45,1,1,2,2,2,2,1,1,2,3,42.30,1840.75,1,0,0,0,0,0
4,0,2,2,2,1,2,1,1,1,1,1,1,70.70,151.65,0,0,0,1,1,1


In [162]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Setting target and features
target = 'Churn'

# Dropping 'result' and 'age' as they seem to be confounding and not helpful
features = df.columns.drop([target])

# Setting X and y
X = df[features]
y = df[target]

# Train test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

# Scaling the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Checking data
X_train[0]

(5634, 19) (5634,) (1409, 19) (1409,)


array([-4.37749204e-01, -4.65683364e-01, -5.07614974e-01, -1.18234670e+00,
        3.55775250e-01,  1.81845392e-01,  2.90697062e-01, -9.05426063e-01,
       -1.07437424e+00,  2.35570413e-01,  3.72908354e-01, -2.78517888e-01,
       -4.73723375e-04, -4.21730250e-01, -1.02516569e+00, -9.69578591e-01,
        1.53218588e+00,  3.29573443e-01, -1.20000722e+00])

In [152]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Important Hyperparameters
inputs = X_train.shape[1]
epochs = 75
batch_size = 10


# Create Model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(inputs,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))

# Compile Model
model.compile(optimizer='adam', loss='mse', metrics=['mse', 'mae'])

# Fit Model
model.fit(X_train, y_train, 
          validation_data=(X_test,y_test), 
          epochs=epochs, 
          batch_size=batch_size
         )

Train on 5634 samples, validate on 1409 samples
Epoch 1/75
5634/5634 [==============================] - 1s 170us/sample - loss: nan - mse: nan - mae: nan - val_loss: nan - val_mse: nan - val_mae: nan
Epoch 2/75
5634/5634 [==============================] - 1s 108us/sample - loss: nan - mse: nan - mae: nan - val_loss: nan - val_mse: nan - val_mae: nan
Epoch 3/75
5634/5634 [==============================] - 1s 110us/sample - loss: nan - mse: nan - mae: nan - val_loss: nan - val_mse: nan - val_mae: nan
Epoch 4/75
5634/5634 [==============================] - 1s 111us/sample - loss: nan - mse: nan - mae: nan - val_loss: nan - val_mse: nan - val_mae: nan
Epoch 5/75
5634/5634 [==============================] - 1s 110us/sample - loss: nan - mse: nan - mae: nan - val_loss: nan - val_mse: nan - val_mae: nan
Epoch 6/75
5634/5634 [==============================] - 1s 112us/sample - loss: nan - mse: nan - mae: nan - val_loss: nan - val_mse: nan - val_mae: nan
Epoch 7/75
5634/5634 [==================

### Using GridSearchCV to tune hyperparameters

In [173]:
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# Load data
url = 'https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv'
data = pd.read_csv(url)
data = wrangle_data(data)
# data.shape
# data.head()

# split into input (X) and output (Y) variables
X = data.iloc[:, 0:18]
y = data.iloc[:, 19]

# Function to create model, required for KerasClassifier
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=18, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
param_grid = {'batch_size': [10, 20, 40, 60, 80, 100],
              'epochs': [20]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X, y)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

Best: 0.7346286535263061 using {'batch_size': 10, 'epochs': 20}
Means: 0.7346286535263061, Stdev: 0.004757632470334884 with: {'batch_size': 10, 'epochs': 20}
Means: 0.7346286535263061, Stdev: 0.004757632470334884 with: {'batch_size': 20, 'epochs': 20}
Means: 0.7346286535263061, Stdev: 0.004757632470334884 with: {'batch_size': 40, 'epochs': 20}
Means: 0.7346286535263061, Stdev: 0.004757632470334884 with: {'batch_size': 60, 'epochs': 20}
Means: 0.7346286535263061, Stdev: 0.004757632470334884 with: {'batch_size': 80, 'epochs': 20}
Means: 0.7346286535263061, Stdev: 0.004757632470334884 with: {'batch_size': 100, 'epochs': 20}


### Trying with different params + optimizers

In [176]:
from keras import optimizers

def optimized_model():
    # Create model
    model = Sequential()
    model.add(Dense(12, input_dim=18, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    # Optimizer
    # sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

    # Compile model
    model.compile(
        loss='binary_crossentropy',
        optimizer='sgd',
        metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=optimized_model, verbose=0)


# define the grid search parameters
param_grid = {'batch_size': [20],
              'epochs': [20, 40, 60, 80, 100, 200, 300]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X, y)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.7346286535263061 using {'batch_size': 20, 'epochs': 20}
Means: 0.7346286535263061, Stdev: 0.004757632470334884 with: {'batch_size': 20, 'epochs': 20}
Means: 0.7346286535263061, Stdev: 0.004757632470334884 with: {'batch_size': 20, 'epochs': 40}
Means: 0.7346286535263061, Stdev: 0.004757632470334884 with: {'batch_size': 20, 'epochs': 60}
Means: 0.7346286535263061, Stdev: 0.004757632470334884 with: {'batch_size': 20, 'epochs': 80}
Means: 0.7346286535263061, Stdev: 0.004757632470334884 with: {'batch_size': 20, 'epochs': 100}
Means: 0.7346286535263061, Stdev: 0.004757632470334884 with: {'batch_size': 20, 'epochs': 200}
Means: 0.7346286535263061, Stdev: 0.004757632470334884 with: {'batch_size': 20, 'epochs': 300}


## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?